<a href="https://colab.research.google.com/github/cars1015/recsys_MyReserch/blob/main/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Googleドライブをマウント
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder

データの選択

In [ ]:
data="ml-20m"
dir = "/content/drive/MyDrive/recommend/" + data+"/i2v_sg/"
df=pd.read_csv(dir+"train_i2v.csv")
print(df)
print(df['sid'].unique())
print(len(df['sid'].unique()))
#word2vecで使用するため文字列に変換
df['sid']=df['sid'].astype(str)

            uid   sid                                            title
0        116149     0                                   Rob Roy (1995)
1        116149     1                               Stand by Me (1986)
2        116149     2                                   Platoon (1986)
3        116149     3                              Shining, The (1980)
4        116149     4  Harry Potter and the Prisoner of Azkaban (2004)
...         ...   ...                                              ...
8538841    8789   932                      Little Miss Sunshine (2006)
8538842    8789  1747                                        Up (2009)
8538843    8789   151                      Nutty Professor, The (1996)
8538844    8789  1776                                Robin Hood (1973)
8538845    8789   592            Fear and Loathing in Las Vegas (1998)

[8538846 rows x 3 columns]
[    0     1     2 ... 20105 20106 20107]
20108


In [ ]:
#ユーザごとにアクションを起こしたアイテムのidをリストにした。
sentences=df.groupby('uid')['sid'].apply(list).tolist()
def find_longest_list(lst):
    max_len = 0
    max_list = None
    
    for sub_list in lst:
        if len(sub_list) > max_len:
            max_len = len(sub_list)
            max_list = sub_list
    
    return len(max_list)
x=find_longest_list(sentences)
print(x)


3177


モデルの作成

In [ ]:
from gensim.models import Word2Vec
#sg=1とすることでitem2vecにしている。
model=Word2Vec(sentences,sg=1,size=300,window=1000,hs=0,negative=5,seed=0,min_count=1)
model.save("i2v.model")

In [ ]:
similarity=model.wv.similarity(str(1),str(2))
print(similarity)
#model.wv.vocab.keys()
model.save(dir+"model")

0.86068016


アイテムごとにベクトルを取り出し、類似度を計算し、アイテム×アイテム行列を作成する。

In [ ]:
size=len(df['sid'].unique())
#アイテムのインデックスを文字列で作成
item_id=df['sid'].unique().astype(str)
similarity_matrix=np.zeros((size,size))

#同アイテムの類似度が1であることは自明なので対角成分の類似度を0に設定
for i in range((size)):
  similarity_matrix[i][i]=0
#アイテム×アイテム行列の作成(類似度はコサイン類似度を用いている。)
for i in range(size):
  for j in range(i+1,size):
    print(i)
    print(j)
    similarity=model.wv.similarity(str(i),str(j))
    similarity_matrix[i][j]=similarity
    similarity_matrix[j][i]=similarity
np.savetxt(dir+"similarity_i2v.txt",similarity_matrix)


ストリーミング出力は最後の 5000 行に切り捨てられました。
554
6308
554
6309
554
6310
554
6311
554
6312
554
6313
554
6314
554
6315
554
6316
554
6317
554
6318
554
6319
554
6320
554
6321
554
6322
554
6323
554
6324
554
6325
554
6326
554
6327
554
6328
554
6329
554
6330
554
6331
554
6332
554
6333
554
6334
554
6335
554
6336
554
6337
554
6338
554
6339
554
6340
554
6341
554
6342
554
6343
554
6344
554
6345
554
6346
554
6347
554
6348
554
6349
554
6350
554
6351
554
6352
554
6353
554
6354
554
6355
554
6356
554
6357
554
6358
554
6359
554
6360
554
6361
554
6362
554
6363
554
6364
554
6365
554
6366
554
6367
554
6368
554
6369
554
6370
554
6371
554
6372
554
6373
554
6374
554
6375
554
6376
554
6377
554
6378
554
6379
554
6380
554
6381
554
6382
554
6383
554
6384
554
6385
554
6386
554
6387
554
6388
554
6389
554
6390
554
6391
554
6392
554
6393
554
6394
554
6395
554
6396
554
6397
554
6398
554
6399
554
6400
554
6401
554
6402
554
6403
554
6404
554
6405
554
6406
554
6407
554
6408
554
6409
554
6410
554
6411
554
6412
554
6413
554
6414
554
6

In [ ]:
class Test:
    def __init__(self):
      #LabelEncoder()は文字列を数値に変えてくれる
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df.loc[:, 'uid'])
        items = self.item_enc.fit_transform(df.loc[:, 'sid'])
        return users, items

    def fit(self, df):
        users, items = self._get_users_and_items(df)
        sentences=df.groupby('uid')['sid'].apply(list).tolist()
        sentences=list(map(str,sentences))

学習したモデルとtest_tr.csvを用い予測

In [ ]:
df_test=pd.read_csv(dir+"test_tr.csv")
model=Test()
model.fit(df)
users = df_test.loc[:, 'uid']
items = df_test.loc[:, 'sid']
u_enc = LabelEncoder()
item_enc=LabelEncoder()
users_id = u_enc.fit_transform(users)
items_id = model.item_enc.transform(items)
items=model.item_enc.transform(items)
values = np.ones(df_test.shape[0])
shape = (u_enc.classes_.size, model.item_enc.classes_.size)
X = csr_matrix((values, (users_id, items_id)), shape=shape)

pred = X.dot(similarity_matrix)

In [ ]:
np.savetxt(dir+"pred_i2v.txt",pred)